In [1]:
!apt-get install sqlite3


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.3 [768 kB]
Fetched 768 kB in 1s (1,204 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.3_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.3) ...
Setting up sqlite3 (3.37.2-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [6]:
import pandas as pd
import sqlite3

# Load the dataset into a Pandas DataFrame
data = pd.read_csv('Customer_data.csv')

# Connect to SQLite (creates a new database if it doesn't exist)
conn = sqlite3.connect('customer_segmentation.db')

# Load the DataFrame into SQLite
data.to_sql('customers', conn, if_exists='replace', index=False)

# Verify the table was created
query = "SELECT * FROM customers LIMIT 5;"
print(pd.read_sql(query, conn))


   Customer ID  Gender  Age           City Membership Type  Total Spend  \
0          101  Female   29       New York            Gold      1120.20   
1          102    Male   34    Los Angeles          Silver       780.50   
2          103  Female   43        Chicago          Bronze       510.75   
3          104    Male   30  San Francisco            Gold      1480.30   
4          105    Male   27          Miami          Silver       720.40   

   Items Purchased  Average Rating  Discount Applied  \
0               14             4.6                 1   
1               11             4.1                 0   
2                9             3.4                 1   
3               19             4.7                 0   
4               13             4.0                 1   

   Days Since Last Purchase Satisfaction Level  
0                        25          Satisfied  
1                        18            Neutral  
2                        42        Unsatisfied  
3               

In [11]:
query = """
SELECT
    COUNT(*) AS Total_Rows,
    SUM(CASE WHEN `Customer ID` IS NULL THEN 1 ELSE 0 END) AS Null_Customer_ID, -- Enclose 'Customer ID' in backticks
    SUM(CASE WHEN `Total Spend` IS NULL THEN 1 ELSE 0 END) AS Null_Total_Spend -- Enclose 'Total Spend' in backticks
FROM customers;
"""
print(pd.read_sql(query, conn))


   Total_Rows  Null_Customer_ID  Null_Total_Spend
0         350                 0                 0


In [13]:
query = """
DELETE FROM customers
WHERE `Customer ID` IS NULL OR `Total Spend` IS NULL; -- Enclose column names with spaces in backticks
"""
conn.execute(query)


In [17]:
query = """
SELECT
    `Customer ID`, -- Enclose column name with backticks
    SUM(`Total Spend`) AS `Total Spend`, -- Enclose column name with backticks
    SUM(`Items Purchased`) AS `Items Purchased`, -- Enclose column name with backticks
    AVG(`Days Since Last Purchase`) AS `Avg Days Since Last Purchase`, -- Enclose the correct column name 'Days Since Last Purchase' with backticks
    MAX(`Satisfaction Level`) AS `Satisfaction Level` -- Enclose column name with backticks
FROM customers
GROUP BY `Customer ID`; -- Enclose column name with backticks
"""
aggregated_data = pd.read_sql(query, conn)
print(aggregated_data.head())


   Customer ID  Total Spend  Items Purchased  Avg Days Since Last Purchase  \
0          101      1120.20               14                          25.0   
1          102       780.50               11                          18.0   
2          103       510.75                9                          42.0   
3          104      1480.30               19                          12.0   
4          105       720.40               13                          55.0   

  Satisfaction Level  
0          Satisfied  
1            Neutral  
2        Unsatisfied  
3          Satisfied  
4        Unsatisfied  


In [20]:
query = """
SELECT *,
    CASE
        WHEN `Total Spend` > 1000 AND `Items Purchased` > 50 THEN 'High Value'  -- Enclose 'Total Spend' and 'Items Purchased' in backticks
        WHEN `Total Spend` BETWEEN 500 AND 1000 THEN 'Moderate Value' -- Enclose 'Total Spend' in backticks
        ELSE 'Low Value'
    END AS `Customer Segment` -- Enclose 'Customer Segment' in backticks to avoid syntax error
FROM (
    SELECT
        `Customer ID`, -- Enclose 'Customer ID' in backticks
        SUM(`Total Spend`) AS `Total Spend`, -- Enclose 'Total Spend' in backticks
        SUM(`Items Purchased`) AS `Items Purchased`, -- Enclose 'Items Purchased' in backticks
        AVG(`Days Since Last Purchase`) AS `Avg Days Since Last Purchase`, -- Enclose 'Days Since Last Purchase' in backticks
        MAX(`Satisfaction Level`) AS `Satisfaction Level` -- Enclose 'Satisfaction Level' in backticks
    FROM customers
    GROUP BY `Customer ID` -- Enclose 'Customer ID' in backticks
) AS aggregated;
"""
segmented_data = pd.read_sql(query, conn)
print(segmented_data.head())


   Customer ID  Total Spend  Items Purchased  Avg Days Since Last Purchase  \
0          101      1120.20               14                          25.0   
1          102       780.50               11                          18.0   
2          103       510.75                9                          42.0   
3          104      1480.30               19                          12.0   
4          105       720.40               13                          55.0   

  Satisfaction Level Customer Segment  
0          Satisfied        Low Value  
1            Neutral   Moderate Value  
2        Unsatisfied   Moderate Value  
3          Satisfied        Low Value  
4        Unsatisfied   Moderate Value  


In [21]:
segmented_data.to_csv('Customer_Segments_SQL.csv', index=False)
